
# Get Dataset

*  We get dataset from google drive that we will use. 
*  We combined all teams in a one directory divided training and test folders.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/My\ Drive/Development_Dataset.zip

In [ ]:
import shutil, random, json

path = '/content/file'
if not os.path.exists(path):       
    if os.path.exists(path):
        shutil.rmtree(path)
    if not os.path.exists(path):
        os.mkdir(path)
    if not os.path.exists(path+'/training'):
        os.mkdir(path+'/training')
    if not os.path.exists(path+'/test'):
        os.mkdir(path+'/test')

    source_image = ['/content/Development_Dataset/Training/Bipbip/Haricot/Images','/content/Development_Dataset/Training/Bipbip/Mais/Images'
                    ,'/content/Development_Dataset/Training/Pead/Haricot/Images','/content/Development_Dataset/Training/Pead/Mais/Images'
                    ,'/content/Development_Dataset/Training/Roseau/Haricot/Images','/content/Development_Dataset/Training/Roseau/Mais/Images'
                    ,'/content/Development_Dataset/Training/Weedelec/Haricot/Images','/content/Development_Dataset/Training/Weedelec/Mais/Images']

    source_mask = ['/content/Development_Dataset/Training/Bipbip/Haricot/Masks','/content/Development_Dataset/Training/Bipbip/Mais/Masks'
                    ,'/content/Development_Dataset/Training/Pead/Haricot/Masks','/content/Development_Dataset/Training/Pead/Mais/Masks'
                    ,'/content/Development_Dataset/Training/Roseau/Haricot/Masks','/content/Development_Dataset/Training/Roseau/Mais/Masks'
                    ,'/content/Development_Dataset/Training/Weedelec/Haricot/Masks','/content/Development_Dataset/Training/Weedelec/Mais/Masks']
    
    source_test = ['/content/Development_Dataset/Test_Dev/Bipbip/Haricot/Images','/content/Development_Dataset/Test_Dev/Bipbip/Mais/Images'
                    ,'/content/Development_Dataset/Test_Dev/Pead/Haricot/Images','/content/Development_Dataset/Test_Dev/Pead/Mais/Images'
                    ,'/content/Development_Dataset/Test_Dev/Roseau/Haricot/Images','/content/Development_Dataset/Test_Dev/Roseau/Mais/Images'
                    ,'/content/Development_Dataset/Test_Dev/Weedelec/Haricot/Images','/content/Development_Dataset/Test_Dev/Weedelec/Mais/Images']

    # Destination paths
    dest_train = path+'/training'
    dest_test = path+'/test'

    if not os.path.exists(dest_train+'/'+'images'):
        os.mkdir(dest_train+'/'+'images')
    if not os.path.exists(dest_train+'/'+'masks'):
        os.mkdir(dest_train+'/'+'masks')
    if not os.path.exists(dest_test+'/'+'images'):
        os.mkdir(dest_test+'/'+'images')
    
    for j in range(int(len(source_image))):
      source_images = source_image[j]
      source_masks = source_mask[j]
      files = os.listdir(source_images)
      m_files = os.listdir(source_masks)
      random.shuffle(files)
      #create training set randomly
      for i in range(int(len(files))):
          dest = shutil.copy(source_images+'/'+files[i], dest_train+'/images/'+files[i])
          dest = shutil.copy(source_masks+'/'+m_files[i], dest_train+'/masks/'+m_files[i])

    for j in range(int(len(source_test))):
      source_img = source_test[j]
      fil = os.listdir(source_img)
      random.shuffle(fil)
      #create training set randomly
      for i in range(int(len(fil))):
          dest = shutil.copy(source_img+'/'+fil[i], dest_test+'/images/'+fil[i])


In [ ]:
input_dir = "/content/file/training/images"
target_dir = "/content/file/training/masks"
img_size = (256, 256)
num_classes = 3
batch_size = 2

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))


# Build Multi-class Image Segmentation

*   We splits 0.25 of the training datas as validation datas.
*   After,we defined train and valid generators and datasets.
*   We defined image size as 256,256 and batch size as 32.
*   We have 3 classes that are weed, crop and background.

*   Then we create CustomDataset inheriting from tf.keras.utils.Sequence that returns a single sample from the dataset

In [ ]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image



class CustomDataset(keras.utils.Sequence):

    """ 
        CustomDataset inheriting from tf.keras.utils.Sequence. Helper to iterate over the data (as Numpy arrays).

        3 main methods:
          - __init__: save dataset params.
          - __len__: return the total number of samples in the dataset into batches.
          - __getitem__: return a sample from the dataset into batches.
          
        Note: 
          - the custom dataset return a single sample from the dataset. Then, we use 
            a tf.data.Dataset object to group samples.

    """

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
        return x, y

In [ ]:
# Split our img paths into a training and a validation set
val_samples = int((len(input_img_paths)*25)/100)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = CustomDataset(batch_size, img_size, train_input_img_paths, train_target_img_paths)
valid_gen = CustomDataset(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [ ]:

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,256, 256, 3], [None,256, 256, 1]))


train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,256, 256, 3], [None,256, 256, 1]))

valid_dataset = valid_dataset.repeat()

In [ ]:
# Let's test data generator
# -------------------------
import time
from matplotlib import cm
import matplotlib.pyplot as plt

%matplotlib inline

# Assign a color to each class
evenly_spaced_interval = np.linspace(0, 1, 20)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

iterator = iter(valid_dataset)
fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)
augmented_img = augmented_img[0]   # First element
augmented_img = augmented_img  # denormalize

target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)

print(np.unique(target))

target_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [254, 124, 18]
target_img[np.where(target == 1)] = [255, 255, 255]
target_img[np.where(target == 2)] = [216, 67, 82]

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))

plt.show()

# Prepare the model for training

*   Here we decide to implement a transfer learning in a model that is VGG-19 which is recommended for segmentation problems. (Encoder-Decoder Network)
*   SparseCategoricalCrossentropy defined for the loss to to use integers (mask) instead of one-hot encoded labels
*   Adam optimizer with learning rate as 1e-4
*   IoU metric



In [ ]:
vgg = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
vgg.summary()
for layer in vgg.layers:
  layer.trainable = False

def create_model(depth,num_classes):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    model.add(vgg)
    
    start_f = 256
        
    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

model = create_model(depth=5, 
                     num_classes=num_classes)

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Visualize created model as a table
model.summary()

# Visualize initialized weights
# model.weights

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


# Training with callbacks
To have better scores we saved the weights, and re-train the model using the old weights. The result has been obtained in this way:

*   Early Stopping,Model checkpoint and Tensorboard were declared as callbacks
*   Training the model 50 times.
*   We reach a plateau. The validation score was stable.

In [ ]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'drive/My Drive/Keras4/', 'multiclass_segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

model.fit(x=train_dataset,
          epochs=50,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)


# Prepare Test to Prediction

*   Firstly, We test model using valid_dataset looking real, target and prediction images.
*   Then, we test model using test images same as before. We created json data and filled our predictions.







In [ ]:
from PIL import Image, ImageOps

# Generate predictions for all images in the validation set
val_gen = CustomDataset(batch_size, img_size, val_input_img_paths, val_target_img_paths)
val_preds = model.predict(val_gen)


def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
    display(img)


# Display results for validation image #10
i = 10

# Display ground-truth target mask
img = ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)

# Display mask predicted by our model
display_mask(i)  # Note that the model only sees inputs at 150x150.

In [ ]:
import time
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
import time
from matplotlib import cm
import matplotlib.pyplot as plt

%matplotlib inline

# Assign a color to each class
evenly_spaced_interval = np.linspace(0, 1, 20)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

iterator = iter(valid_dataset)
fig, ax = plt.subplots(1, 3, figsize=(8, 8))
fig.show()
image, target = next(iterator)

image = image[0]
target = target[0, ..., 0]

out_sigmoid = model.predict(x=tf.expand_dims(image, 0))

# Get predicted class as the index corresponding to the maximum value in the vector probability
# predicted_class = tf.cast(out_sigmoid > score_th, tf.int32)
# predicted_class = predicted_class[0, ..., 0]
predicted_class = tf.argmax(out_sigmoid, -1)

out_sigmoid.shape

predicted_class = predicted_class[0, ...]

# Assign colors (just for visualization)
target_img = np.zeros([target.shape[0], target.shape[1], 3])
prediction_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [254 ,124 ,18]
target_img[np.where(target == 1)] = [255, 255, 255]
target_img[np.where(target == 2)] = [216, 67, 82]


prediction_img[np.where(predicted_class == 0)] = [254 ,124 ,18]
prediction_img[np.where(predicted_class == 1)] = [255, 255, 255]
prediction_img[np.where(predicted_class == 2)] = [216, 67, 82]

ax[0].imshow(np.uint8(image))
ax[1].imshow(np.uint8(target_img))
ax[2].imshow(np.uint8(prediction_img))

fig.canvas.draw()
time.sleep(1)

In [ ]:
# Cycle over test images    
test_dir = "/content/file/test"
test_img_dir = os.path.join(test_dir, 'images')

img_filenames = os.listdir(test_img_dir)


def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)   

evenly_spaced_interval = np.linspace(0, 1, 20)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

# Finally, save the results into the submission.json file
with open('/content/file/submission.json', 'w') as f:
  for img_filename in img_filenames:
      img_name = img_filename
      img = Image.open(os.path.join(test_img_dir,img_name))
      img = img.resize((256, 256))
      img_arr = np.expand_dims(np.array(img), 0)

      out_sigmoid = model.predict(x=img_arr / 1.)

      # Get predicted class as the index corresponding to the maximum value in the vector probability
      # predicted_class = tf.cast(out_sigmoid > score_th, tf.int32)
      # predicted_class = predicted_class[0, ..., 0]
      predicted_class = tf.argmax(out_sigmoid, -1)

      predicted_class = predicted_class[0, ...]
      prediction_img[np.where(predicted_class == 0)] = [254 ,124 ,18]
      prediction_img[np.where(predicted_class == 1)] = [255, 255, 255]
      prediction_img[np.where(predicted_class == 2)] = [216, 67, 82]
        

      t=img_name.split('_')

      submission_dict = {}
      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = '[256,256]'
      submission_dict[img_name]['team'] = t[0]
      submission_dict[img_name]['crop'] = t[1]
      submission_dict[img_name]['segmentation'] = {}

      # crop
      rle_encoded_crop = rle_encode( np.uint8(prediction_img[np.where(predicted_class == 1)]/255) )
      # weed
      rle_encoded_weed = rle_encode( np.uint8(prediction_img[np.where(predicted_class == 2)]/255) )

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed
      json.dump(submission_dict, f)